In [4]:
# Import dependencies.
import matplotlib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
engine = create_engine("sqlite:///open_university.sqlite", echo=False)

## Load the data

The plan is to just look at the interaction with the course material to see if the student will be able to predict if the student will pass or fail.

Not including information about the background of the student (demographics) or the scores of the student assessments but include if they submit the assessments or not.

We are going to filter out students that withdraw from the course before the start of the course.

In [14]:
student_ds = engine.execute(text("""
SELECT sI.code_module, sI.code_presentation, sI.id_student, sI.final_result, sR.date_registration, sR.date_unregistration
FROM studentInfo as sI
LEFT JOIN studentRegistration as sR ON sI.id_student = sR.id_student
WHERE NOT (sR.date_unregistration <= -11 AND sI.final_result = 'Withdrawn')
""")).fetchall()
student_df = pd.DataFrame(student_ds, columns=['code_module', 'code_presentation', 'id_student', 'final_result', 'date_registration', 'date_unregistration'])
student_df = student_df.astype({'code_module':'string', 'code_presentation':'string', 'id_student':'string', 'final_result':'string'})

In [26]:
display(student_df)
display(student_df.describe())
display(student_df.nunique())
display(student_df['final_result'].value_counts())
student_df.info()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
0,AAA,2013J,11391,Pass,-159.0,NaN
1,AAA,2013J,28400,Pass,-53.0,NaN
2,AAA,2013J,30268,Withdrawn,-92.0,12.0
3,AAA,2013J,31604,Pass,-52.0,NaN
4,AAA,2013J,32885,Pass,-176.0,NaN
...,...,...,...,...,...,...
36683,GGG,2014J,2640965,Fail,-4.0,NaN
36684,GGG,2014J,2645731,Distinction,-23.0,NaN
36685,GGG,2014J,2648187,Pass,-129.0,NaN
36686,GGG,2014J,2679821,Withdrawn,-49.0,101.0


,date_registration,date_unregistration
count,36666.000000,11592.000000
mean,-68.496073,73.590666
std,49.179962,73.058458
min,-320.000000,-365.000000
25%,-99.000000,12.000000
50%,-54.000000,55.000000
75%,-29.000000,131.000000
max,167.000000,444.000000


code_module                7
code_presentation          4
id_student             27295
final_result               4
date_registration        320
date_unregistration      355
dtype: int64

final_result
Pass           14682
Withdrawn       9904
Fail            8478
Distinction     3624
Name: count, dtype: Int64

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
716,AAA,2014J,2318055,Pass,-56.0,-19.0
987,BBB,2013B,335910,Fail,-184.0,-43.0
1816,BBB,2013B,543356,Pass,-52.0,-23.0
2812,BBB,2013J,393327,Pass,-128.0,-109.0
3271,BBB,2013J,546941,Distinction,-53.0,-30.0
...,...,...,...,...,...,...
35251,GGG,2014B,542562,Pass,-225.0,-212.0
35253,GGG,2014B,542562,Pass,-85.0,-73.0
35258,GGG,2014B,548578,Fail,-89.0,-87.0
35639,GGG,2014B,625903,Pass,-78.0,-18.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36688 entries, 0 to 36687
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          36688 non-null  string 
 1   code_presentation    36688 non-null  string 
 2   id_student           36688 non-null  string 
 3   final_result         36688 non-null  string 
 4   date_registration    36666 non-null  float64
 5   date_unregistration  11592 non-null  float64
dtypes: float64(2), string(4)
memory usage: 1.7 MB


* I removed students that Withdrew before 11 days before the presentation starts because they whould not have participated in the course.
* It looks like out of the 27,295 students there were 36,388 registrations which means some students where registered more than once. Of all the registrations there where 11,592 unregistrations.
* 


In [38]:
bf_course_df = student_df.loc[student_df['date_unregistration'] <= -11].copy()
display(bf_course_df.head())
display(bf_course_df.describe())
display(student_df.loc[student_df['date_registration'].isnull()])
bf_course_df.info()

,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
716,AAA,2014J,2318055,Pass,-56.0,-19.0
987,BBB,2013B,335910,Fail,-184.0,-43.0
1816,BBB,2013B,543356,Pass,-52.0,-23.0
2812,BBB,2013J,393327,Pass,-128.0,-109.0
3271,BBB,2013J,546941,Distinction,-53.0,-30.0


,date_registration,date_unregistration
count,246.000000,252.000000
mean,-117.556911,-66.781746
std,59.090201,50.987812
min,-320.000000,-365.000000
25%,-148.000000,-87.500000
50%,-115.000000,-53.000000
75%,-78.000000,-26.000000
max,-17.000000,-11.000000


,code_module,code_presentation,id_student,final_result,date_registration,date_unregistration
2339,BBB,2013B,630346,Fail,NaN,NaN
2530,BBB,2013J,57369,Withdrawn,NaN,-1.0
11340,CCC,2014J,394791,Pass,NaN,-61.0
14132,CCC,2014J,1777834,Pass,NaN,NaN
15825,DDD,2013B,2707979,Fail,NaN,NaN
15826,DDD,2013B,2710343,Fail,NaN,NaN
15827,DDD,2013B,2710343,Fail,NaN,NaN
15921,DDD,2013J,128965,Pass,NaN,-24.0
17651,DDD,2013J,604398,Withdrawn,NaN,-1.0
18802,DDD,2014B,604398,Withdrawn,NaN,-1.0


<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 716 to 35732
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          252 non-null    string 
 1   code_presentation    252 non-null    string 
 2   id_student           252 non-null    string 
 3   final_result         252 non-null    string 
 4   date_registration    246 non-null    float64
 5   date_unregistration  252 non-null    float64
dtypes: float64(2), string(4)
memory usage: 13.8 KB


In [46]:
## need to add the Vle connected to Student Vle
vle_ds = engine.execute(text("""
SELECT cast(vle.id_site as text), vle.code_module, vle.code_presentation, vle.activity_type, vle.week_from, vle.week_to, cast(sVle.id_student as text), sVle.date, SUM(sVLe.sum_click) as sum_click
FROM vle
LEFT JOIN studentVle AS sVle ON vle.id_site = sVle.id_site AND vle.code_presentation = sVle.code_presentation AND vle.code_module = sVle.code_module
GROUP BY vle.code_module, vle.code_presentation, sVle.id_student
""")).fetchall()
vle_df = pd.DataFrame(vle_ds, columns=['id_site', 'code_module', 'code_presentation', 'activity_type', 'week_from', 'week_to', 'id_student', 'date', 'sum_click'])
vle_df = vle_df.astype({'id_site':'string', 'code_module':'string', 'code_presentation':'string', 'activity_type':'string', 'id_student':'string'})

In [47]:
display(vle_df)
display(vle_df.nunique())
vle_df.info()

,id_site,code_module,code_presentation,activity_type,week_from,week_to,id_student,date,sum_click
0,546897,AAA,2013J,url,NaN,NaN,<NA>,NaN,NaN
1,546614,AAA,2013J,homepage,NaN,NaN,11391,-5.0,934.0
2,546614,AAA,2013J,homepage,NaN,NaN,28400,-10.0,1435.0
3,546614,AAA,2013J,homepage,NaN,NaN,30268,-10.0,281.0
4,546614,AAA,2013J,homepage,NaN,NaN,31604,-10.0,2158.0
...,...,...,...,...,...,...,...,...,...
29240,897051,GGG,2014J,resource,2.0,2.0,2640965,17.0,41.0
29241,896962,GGG,2014J,oucontent,NaN,NaN,2645731,110.0,893.0
29242,896956,GGG,2014J,quiz,NaN,NaN,2648187,149.0,312.0
29243,897051,GGG,2014J,resource,2.0,2.0,2679821,-6.0,275.0


id_site                623
code_module              7
code_presentation        4
activity_type           15
week_from               24
week_to                 24
id_student           26074
date                   281
sum_click             5380
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29245 entries, 0 to 29244
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            29245 non-null  string 
 1   code_module        29245 non-null  string 
 2   code_presentation  29245 non-null  string 
 3   activity_type      29245 non-null  string 
 4   week_from          4705 non-null   float64
 5   week_to            4705 non-null   float64
 6   id_student         29228 non-null  string 
 7   date               29228 non-null  float64
 8   sum_click          29228 non-null  float64
dtypes: float64(4), string(5)
memory usage: 2.0 MB


In [48]:
ass_ds = engine.execute(text("""
SELECT cast(sAss.id_student as text), ass.code_module, ass.code_presentation, cast(sAss.id_assessment as text), sAss.date_submitted, ass.date
FROM studentAssessment as sAss
LEFT JOIN assessments as ass ON sAss.id_assessment = ass.id_assessment
""")).fetchall()

assessment_df = pd.DataFrame(ass_ds, columns=['id_student', 'code_module', 'code_presentation', 'id_assessent', 'date_submitted', 'date'])
display(assessment_df.describe())
assessment_df.info()

,date_submitted,date
count,173912.000000,171047.000000
mean,116.032942,130.605623
std,71.484148,78.025175
min,-11.000000,12.000000
25%,51.000000,54.000000
50%,116.000000,129.000000
75%,173.000000,214.000000
max,608.000000,261.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_student         173912 non-null  object 
 1   code_module        173912 non-null  object 
 2   code_presentation  173912 non-null  object 
 3   id_assessent       173912 non-null  object 
 4   date_submitted     173912 non-null  int64  
 5   date               171047 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 8.0+ MB


In [ ]:
#df = df.astype({'gender':'string', 'imd_band':'string', 'highest_education':'string', 'age_band':'string', 'region':'string', 'final_result':'string'})
#df.dtypes


In [ ]:
display(df.nunique())
#display(df.value_counts("gender"))
#display(df.value_counts("imd_band"))
#display(df.value_counts("highest_education"))
#print(f"A-Level is equivilent to high school cert, HE Qualification is level 1 ro 2 of University, ")
#display(df.value_counts("age_band"))
#display(df.value_counts("num_of_prev_attempts"))
#display(df.value_counts("region"))
display(df.value_counts("final_result"))

In [ ]:
# distinction is a higher form of pass
# withdrawn would be a fail if the student would not complete the course both fail and withdrawn is a form of failure.
# Possible to remove withdrawn
df_edited = df.copy()
df_edited["final_result"] = df_edited["final_result"].replace({'Distinction': 'Pass', 'Withdrawn': 'Fail'})
df_edited["final_result"] = df_edited["final_result"].replace({'Pass': '1', 'Fail': '0'})
df_edited = df_edited.astype({'day': 'float32', 'clicks': 'float32', 'num_of_prev_attempts': 'float32', 'final_result': 'float32'})
display(df_edited["final_result"].unique())
display(df_edited.dtypes)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
df_edited = pd.get_dummies(df_edited)
df_edited.head()

In [ ]:
# combine pass and distinction
# combine widthdrawn and fail
X = df_edited.drop('final_result', axis=1).values
y = df_edited['final_result'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# X shape / input feature / columns rule of thumbs is normally 2-3 times the ammount so 43 * 2 or 3 is 86-129 so i will try choosing 90 for the first hidden node
hidden_nodes1 = 90
hidden_nodes2 = 60
hidden_nodes3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_shape=(number_input_features,), activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=20)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("models/Student_Pass1Fail_relu90+relu60+relu30+sigmoid.h5")

In [ ]:
# plotting the accuracy
df_edited = pd.DataFrame(fit_model.history, index = range(1, len(fit_model.history['loss'])+1))
df_edited.plot(y = 'accuracy')